<a href="https://colab.research.google.com/github/CatarinaL/ddosLogs/blob/master/ddos_logs_logit_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import numpy as np
import pandas as pd
from scipy.io import arff
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [35]:
#/content/drive/MyDrive/datasets/ddosLogs/final-dataset.arff
np.random.seed(123)
file = "/content/drive/MyDrive/datasets/ddosLogs/final-dataset.arff"

data, meta = arff.loadarff(file)
df = pd.DataFrame(data=data)
df.info

<bound method DataFrame.info of          SRC_ADD  DES_ADD  ...  LAST_PKT_RESEVED     PKT_CLASS
0           3.00    24.30  ...         50.021920     b'Normal'
1          15.00    24.15  ...         50.030211     b'Normal'
2          24.15    15.00  ...         50.060221  b'UDP-Flood'
3          24.90     9.00  ...         50.060098     b'Normal'
4          24.80     8.00  ...         50.061864     b'Normal'
...          ...      ...  ...               ...           ...
2160663    24.10    10.00  ...         50.040562     b'Normal'
2160664    24.12    12.00  ...         50.051067     b'Normal'
2160665     1.00    24.10  ...         50.013418     b'Normal'
2160666     5.00    24.50  ...         50.027339     b'Normal'
2160667    15.00    24.15  ...         50.030211     b'Normal'

[2160668 rows x 28 columns]>

In [36]:
df

,SRC_ADD,DES_ADD,PKT_ID,FROM_NODE,TO_NODE,PKT_TYPE,PKT_SIZE,FLAGS,FID,SEQ_NUMBER,NUMBER_OF_PKT,NUMBER_OF_BYTE,NODE_NAME_FROM,NODE_NAME_TO,PKT_IN,PKT_OUT,PKT_R,PKT_DELAY_NODE,PKT_RATE,BYTE_RATE,PKT_AVG_SIZE,UTILIZATION,PKT_DELAY,PKT_SEND_TIME,PKT_RESEVED_TIME,FIRST_PKT_SENT,LAST_PKT_RESEVED,PKT_CLASS
0,3.00,24.30,389693.0,21.0,23.0,b'tcp',1540.0,b'-------',4.0,11339.0,16091.0,24780100.0,b'Switch1',b'Router',35.529786,35.529786,35.539909,0.0,328.240918,505490.0,1540.0,0.236321,0.0,35.519662,35.550032,1.000000,50.021920,b'Normal'
1,15.00,24.15,201196.0,23.0,24.0,b'tcp',1540.0,b'-------',16.0,6274.0,16092.0,24781700.0,b'Router',b'server1',20.176725,20.176725,20.186848,0.0,328.205808,505437.0,1540.0,0.236337,0.0,20.156478,20.186848,1.000000,50.030211,b'Normal'
2,24.15,15.00,61905.0,23.0,22.0,b'ack',55.0,b'-------',16.0,1930.0,16092.0,885060.0,b'Router',b'Switch2',7.049955,7.049955,7.059958,0.0,328.206042,18051.3,55.0,0.008441,0.0,7.039952,7.069962,1.030045,50.060221,b'UDP-Flood'
3,24.90,9.00,443135.0,23.0,21.0,b'ack',55.0,b'-------',10.0,12670.0,16085.0,884675.0,b'Router',b'Switch1',39.627970,39.627970,39.637973,0.0,328.064183,18043.5,55.0,0.008437,0.0,39.617967,39.647976,1.030058,50.060098,b'Normal'
4,24.80,8.00,157335.0,23.0,21.0,b'ack',55.0,b'-------',9.0,4901.0,16088.0,884840.0,b'Router',b'Switch1',16.039806,16.039806,16.049810,0.0,328.113525,18046.2,55.0,0.008438,0.0,16.029803,16.059813,1.030054,50.061864,b'Normal'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160663,24.10,10.00,43184.0,22.0,10.0,b'ack',55.0,b'-------',11.0,1343.0,16103.0,885665.0,b'Switch2',b'clien-10',5.295040,5.295040,5.305043,0.0,328.561918,18070.9,55.0,0.008446,0.0,5.275034,5.305043,1.030013,50.040562,b'Normal'
2160664,24.12,12.00,376386.0,23.0,22.0,b'ack',55.0,b'-------',13.0,10991.0,16103.0,885665.0,b'Router',b'Switch2',34.508649,34.508649,34.518652,0.0,328.491596,18067.0,55.0,0.008446,0.0,34.498646,34.528655,1.030026,50.051067,b'Normal'
2160665,1.00,24.10,140121.0,21.0,23.0,b'tcp',1540.0,b'-------',2.0,4374.0,16091.0,24780100.0,b'Switch1',b'Router',14.418011,14.418011,14.428134,0.0,328.297855,505578.0,1540.0,0.236321,0.0,14.407888,14.438258,1.000000,50.013418,b'Normal'
2160666,5.00,24.50,26800.0,5.0,21.0,b'tcp',1540.0,b'-------',6.0,832.0,16091.0,24780100.0,b'clien-5',b'Switch1',3.732227,3.732227,3.742350,0.0,328.204637,505434.0,1540.0,0.236321,0.0,3.732227,3.762597,1.000000,50.027339,b'Normal'


In [37]:
#decode byte strings to remove the "b" before the srting values at cols PKT_CLASS, NODE_NAME_FROM, NODE_NAME_TO, FLAGS, PKT_TYPE 
df["PKT_CLASS"] = df["PKT_CLASS"].apply(lambda x: x.decode('utf-8')) 
df["NODE_NAME_FROM"] = df["NODE_NAME_FROM"].apply(lambda x: x.decode('utf-8')) 
df["NODE_NAME_TO"] = df["NODE_NAME_TO"].apply(lambda x: x.decode('utf-8')) 
df["FLAGS"] = df["FLAGS"].apply(lambda x: x.decode('utf-8')) 
df["PKT_TYPE"] = df["PKT_TYPE"].apply(lambda x: x.decode('utf-8')) 

df

,SRC_ADD,DES_ADD,PKT_ID,FROM_NODE,TO_NODE,PKT_TYPE,PKT_SIZE,FLAGS,FID,SEQ_NUMBER,NUMBER_OF_PKT,NUMBER_OF_BYTE,NODE_NAME_FROM,NODE_NAME_TO,PKT_IN,PKT_OUT,PKT_R,PKT_DELAY_NODE,PKT_RATE,BYTE_RATE,PKT_AVG_SIZE,UTILIZATION,PKT_DELAY,PKT_SEND_TIME,PKT_RESEVED_TIME,FIRST_PKT_SENT,LAST_PKT_RESEVED,PKT_CLASS
0,3.00,24.30,389693.0,21.0,23.0,tcp,1540.0,-------,4.0,11339.0,16091.0,24780100.0,Switch1,Router,35.529786,35.529786,35.539909,0.0,328.240918,505490.0,1540.0,0.236321,0.0,35.519662,35.550032,1.000000,50.021920,Normal
1,15.00,24.15,201196.0,23.0,24.0,tcp,1540.0,-------,16.0,6274.0,16092.0,24781700.0,Router,server1,20.176725,20.176725,20.186848,0.0,328.205808,505437.0,1540.0,0.236337,0.0,20.156478,20.186848,1.000000,50.030211,Normal
2,24.15,15.00,61905.0,23.0,22.0,ack,55.0,-------,16.0,1930.0,16092.0,885060.0,Router,Switch2,7.049955,7.049955,7.059958,0.0,328.206042,18051.3,55.0,0.008441,0.0,7.039952,7.069962,1.030045,50.060221,UDP-Flood
3,24.90,9.00,443135.0,23.0,21.0,ack,55.0,-------,10.0,12670.0,16085.0,884675.0,Router,Switch1,39.627970,39.627970,39.637973,0.0,328.064183,18043.5,55.0,0.008437,0.0,39.617967,39.647976,1.030058,50.060098,Normal
4,24.80,8.00,157335.0,23.0,21.0,ack,55.0,-------,9.0,4901.0,16088.0,884840.0,Router,Switch1,16.039806,16.039806,16.049810,0.0,328.113525,18046.2,55.0,0.008438,0.0,16.029803,16.059813,1.030054,50.061864,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160663,24.10,10.00,43184.0,22.0,10.0,ack,55.0,-------,11.0,1343.0,16103.0,885665.0,Switch2,clien-10,5.295040,5.295040,5.305043,0.0,328.561918,18070.9,55.0,0.008446,0.0,5.275034,5.305043,1.030013,50.040562,Normal
2160664,24.12,12.00,376386.0,23.0,22.0,ack,55.0,-------,13.0,10991.0,16103.0,885665.0,Router,Switch2,34.508649,34.508649,34.518652,0.0,328.491596,18067.0,55.0,0.008446,0.0,34.498646,34.528655,1.030026,50.051067,Normal
2160665,1.00,24.10,140121.0,21.0,23.0,tcp,1540.0,-------,2.0,4374.0,16091.0,24780100.0,Switch1,Router,14.418011,14.418011,14.428134,0.0,328.297855,505578.0,1540.0,0.236321,0.0,14.407888,14.438258,1.000000,50.013418,Normal
2160666,5.00,24.50,26800.0,5.0,21.0,tcp,1540.0,-------,6.0,832.0,16091.0,24780100.0,clien-5,Switch1,3.732227,3.732227,3.742350,0.0,328.204637,505434.0,1540.0,0.236321,0.0,3.732227,3.762597,1.000000,50.027339,Normal


In [38]:
df.describe()
df.isnull().values.any()

False

In [39]:
df["PKT_CLASS"] = df["PKT_CLASS"].astype('category')
df["NODE_NAME_FROM"] = df["NODE_NAME_FROM"].astype('category') 
df["NODE_NAME_TO"] = df["NODE_NAME_TO"].astype('category') 
df["FLAGS"] = df["FLAGS"].astype('category') 
df["PKT_TYPE"] = df["PKT_TYPE"].astype('category')


In [40]:
df["SRC_ADD"] = df["SRC_ADD"].astype('int64')
df["DES_ADD"] = df["DES_ADD"].astype('int64')
df["PKT_ID"] = df["PKT_ID"].astype('int64')
df["FROM_NODE"] = df["FROM_NODE"].astype('int64')
df["TO_NODE"] = df["TO_NODE"].astype('int64')
df["FID"] = df["FID"].astype('int64')


In [41]:
df.dtypes


SRC_ADD                int64
DES_ADD                int64
PKT_ID                 int64
FROM_NODE              int64
TO_NODE                int64
PKT_TYPE            category
PKT_SIZE             float64
FLAGS               category
FID                    int64
SEQ_NUMBER           float64
NUMBER_OF_PKT        float64
NUMBER_OF_BYTE       float64
NODE_NAME_FROM      category
NODE_NAME_TO        category
PKT_IN               float64
PKT_OUT              float64
PKT_R                float64
PKT_DELAY_NODE       float64
PKT_RATE             float64
BYTE_RATE            float64
PKT_AVG_SIZE         float64
UTILIZATION          float64
PKT_DELAY            float64
PKT_SEND_TIME        float64
PKT_RESEVED_TIME     float64
FIRST_PKT_SENT       float64
LAST_PKT_RESEVED     float64
PKT_CLASS           category
dtype: object

copying the df to one with target dummy variable (y), leaving original intact 

In [51]:
df2 = df.copy()

X = df2.iloc[:, 0:-1]
y = df2.iloc[:, [-1]]

In [52]:
X

,SRC_ADD,DES_ADD,PKT_ID,FROM_NODE,TO_NODE,PKT_TYPE,PKT_SIZE,FLAGS,FID,SEQ_NUMBER,NUMBER_OF_PKT,NUMBER_OF_BYTE,NODE_NAME_FROM,NODE_NAME_TO,PKT_IN,PKT_OUT,PKT_R,PKT_DELAY_NODE,PKT_RATE,BYTE_RATE,PKT_AVG_SIZE,UTILIZATION,PKT_DELAY,PKT_SEND_TIME,PKT_RESEVED_TIME,FIRST_PKT_SENT,LAST_PKT_RESEVED
0,3,24,389693,21,23,tcp,1540.0,-------,4,11339.0,16091.0,24780100.0,Switch1,Router,35.529786,35.529786,35.539909,0.0,328.240918,505490.0,1540.0,0.236321,0.0,35.519662,35.550032,1.000000,50.021920
1,15,24,201196,23,24,tcp,1540.0,-------,16,6274.0,16092.0,24781700.0,Router,server1,20.176725,20.176725,20.186848,0.0,328.205808,505437.0,1540.0,0.236337,0.0,20.156478,20.186848,1.000000,50.030211
2,24,15,61905,23,22,ack,55.0,-------,16,1930.0,16092.0,885060.0,Router,Switch2,7.049955,7.049955,7.059958,0.0,328.206042,18051.3,55.0,0.008441,0.0,7.039952,7.069962,1.030045,50.060221
3,24,9,443135,23,21,ack,55.0,-------,10,12670.0,16085.0,884675.0,Router,Switch1,39.627970,39.627970,39.637973,0.0,328.064183,18043.5,55.0,0.008437,0.0,39.617967,39.647976,1.030058,50.060098
4,24,8,157335,23,21,ack,55.0,-------,9,4901.0,16088.0,884840.0,Router,Switch1,16.039806,16.039806,16.049810,0.0,328.113525,18046.2,55.0,0.008438,0.0,16.029803,16.059813,1.030054,50.061864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160663,24,10,43184,22,10,ack,55.0,-------,11,1343.0,16103.0,885665.0,Switch2,clien-10,5.295040,5.295040,5.305043,0.0,328.561918,18070.9,55.0,0.008446,0.0,5.275034,5.305043,1.030013,50.040562
2160664,24,12,376386,23,22,ack,55.0,-------,13,10991.0,16103.0,885665.0,Router,Switch2,34.508649,34.508649,34.518652,0.0,328.491596,18067.0,55.0,0.008446,0.0,34.498646,34.528655,1.030026,50.051067
2160665,1,24,140121,21,23,tcp,1540.0,-------,2,4374.0,16091.0,24780100.0,Switch1,Router,14.418011,14.418011,14.428134,0.0,328.297855,505578.0,1540.0,0.236321,0.0,14.407888,14.438258,1.000000,50.013418
2160666,5,24,26800,5,21,tcp,1540.0,-------,6,832.0,16091.0,24780100.0,clien-5,Switch1,3.732227,3.732227,3.742350,0.0,328.204637,505434.0,1540.0,0.236321,0.0,3.732227,3.762597,1.000000,50.027339


In [53]:
y

,PKT_CLASS
0,Normal
1,Normal
2,UDP-Flood
3,Normal
4,Normal
...,...
2160663,Normal
2160664,Normal
2160665,Normal
2160666,Normal


In [54]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

#TODO dummy variables w/ WoE


In [ ]:
#TODO normalize

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

In [56]:
#TODO:  instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(X_train,y_train)

#
y_pred=logreg.predict(X_test)

# import the metrics class
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

ValueError: ignored